# Import bibliotek oraz wczytanie danych

In [37]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn.model_selection import train_test_split
import random
import scipy.sparse as sp
from libreco.data import random_split, DatasetPure, split_by_ratio_chrono
from libreco.algorithms import LightGCN
from libreco.evaluation import evaluate

In [38]:
df = pd.read_csv("C:\\Users\\marci\\OneDrive\\Pulpit\\archive\\interactions.csv",sep=',')

## Wstępna obróbka danych

Podział wartości z kolumny `event` tak aby móc dostosować naszą ramkę danych do zadania. 

In [39]:
rating = {
    'click': 1,
    'bookmark': 2,
    'chat_click': 3,
    'contact_phone_click_1': 3,
    'contact_partner_click': 3,
    'contact_phone_click_2': 3,
    'contact_phone_click_3': 3,
    'contact_chat': 3
}
df['event'] = df['event'].map(rating)
df = df.rename(columns={'event': 'label'})
df = df.rename(columns={'timestamp': 'time'})

Ustawiamy liczbę interakcji użytkowników od największej do najmniejszej i sprawdzamy jak wyglądają te liczby.

In [40]:
user_interaction_counts = df['user'].value_counts()

# Sortowanie użytkowników według liczby interakcji od największej do najmniejszej
sorted_users = user_interaction_counts.sort_values(ascending=False)

# Wyświetlenie posortowanych użytkowników
print(sorted_users.head())

user
750207     1310
2358298    1308
3057718    1308
88606      1307
2016978    1303
Name: count, dtype: int64


In [41]:
items_interaction_counts = df['item'].value_counts()

# Sortowanie użytkowników według liczby interakcji od największej do najmniejszej
sorted_items = items_interaction_counts.sort_values(ascending=False)

# Wyświetlenie posortowanych użytkowników
print(sorted_items.head())

item
145769    15480
171495    15092
44153     14581
41518     14544
179685    13985
Name: count, dtype: int64


Pierwsza filtracja ramki danych. Wybieramy 20% przedmiotów o największej liczbie interakcji.

In [42]:
# Obliczenie liczby interakcji dla każdego przedmiotu
item_interaction_counts = df['item'].value_counts()

# Obliczenie kwantyla dla top 20% przedmiotów
top_20_threshold = item_interaction_counts.quantile(0.8)

# Wybór przedmiotów, które są w top 20% o największej liczbie interakcji
top_20_items = item_interaction_counts[item_interaction_counts >= top_20_threshold].index

# Przefiltrowanie oryginalnej ramki danych, aby pozostawić tylko wybranych użytkowników
filtered_df = df[df['item'].isin(top_20_items)]

In [43]:
item_interaction_counts = filtered_df['item'].value_counts()

# Sortowanie użytkowników według liczby interakcji od największej do najmniejszej
sorted_items = item_interaction_counts.sort_values(ascending=False)

# Wyświetlenie posortowanych użytkowników
print(sorted_items.head())

item
145769    15480
171495    15092
44153     14581
41518     14544
179685    13985
Name: count, dtype: int64


To samo robimy dla 20% użytkowników o największej liczbie interakcji.

In [44]:
user_interaction_counts = filtered_df['user'].value_counts()

# Sortowanie użytkowników według liczby interakcji od największej do najmniejszej
sorted_users = user_interaction_counts.sort_values(ascending=False)

# Wyświetlenie posortowanych użytkowników
print(sorted_users.head())

user
2817512    1197
3057718    1190
1288580    1187
610974     1186
1627178    1181
Name: count, dtype: int64


In [45]:
# Obliczenie liczby interakcji dla każdego użytkownika
user_interaction_counts = df['user'].value_counts()

# Obliczenie kwantyla dla top 20% użytkowników
top_20_threshold = user_interaction_counts.quantile(0.8)

# Wybór użytkowników, którzy są w top 20% o największej liczbie interakcji
top_20_users = user_interaction_counts[user_interaction_counts >= top_20_threshold].index

# Przefiltrowanie oryginalnej ramki danych, aby pozostawić tylko wybranych użytkowników
filtered_df = filtered_df[filtered_df['user'].isin(top_20_users)]

In [46]:
item_interaction_counts = filtered_df['item'].value_counts()

# Sortowanie użytkowników według liczby interakcji od największej do najmniejszej
sorted_items = item_interaction_counts.sort_values(ascending=False)

# Wyświetlenie posortowanych użytkowników
print(sorted_items.head())

user_interaction_counts = filtered_df['user'].value_counts()

# Sortowanie użytkowników według liczby interakcji od największej do najmniejszej
sorted_userss = user_interaction_counts.sort_values(ascending=False)

# Wyświetlenie posortowanych użytkowników
print(sorted_users.head())

item
44153     12171
171495    11972
179685    11907
16819     11563
145769    11385
Name: count, dtype: int64
user
2817512    1197
3057718    1190
1288580    1187
610974     1186
1627178    1181
Name: count, dtype: int64


Porównujemy proporcję przedmioty / użytkownicy w wejściowej ramce danych oraz przefiltrowanej. Jak widzimy, proporcja ta jest prawie identyczna.

In [47]:
count_users_before = df['user'].nunique()
count_items_before = df['item'].nunique()
count_users = filtered_df['user'].nunique()
print(count_users)
count_items = filtered_df['item'].nunique()
print(count_items)
print(count_items/count_users * 100)
count_items_before/count_users_before * 100

674019
37125
5.5080049672190246


5.624947283659725

Kolejna filtracja ramki danych. Teraz losowo wybieramy 20% unikatowych przedmiotów i użytkowników tak, aby maszyny, na których pracujemy nie miały problemów z pamięcią.

In [48]:
# Procent użytkowników i przedmiotów do wyboru
sample_percent = 0.20

unique_users_count = filtered_df['user'].nunique()
unique_items_count = filtered_df['item'].nunique()

# Wybór x% unikatowych użytkowników
sampled_users = filtered_df.drop_duplicates('user').sample(frac=sample_percent, random_state=2024)

# Wybór x% unikatowych przedmiotów
sampled_items = filtered_df.drop_duplicates('item').sample(frac=sample_percent, random_state=2024)

# Przefiltrowanie przefiltrowanej ramki danych, aby pozostawić tylko wybrane użytkowniki
filtered_df_users = filtered_df[filtered_df['user'].isin(sampled_users['user'])]

# Przefiltrowanie wynikowej ramki danych, aby pozostawić tylko wybrane przedmioty
filtered_df = filtered_df_users[filtered_df_users['item'].isin(sampled_items['item'])]

Znów sprawdzmy proporcję przedmioty / użytkownicy. 

In [49]:
count_users = filtered_df['user'].nunique()
print(count_users)
count_items = filtered_df['item'].nunique()
print(count_items)
print(count_items/count_users * 100)
count_items_before/count_users_before * 100

130904
7425
5.672095581494836


5.624947283659725

In [50]:
user_interaction_counts = filtered_df['user'].value_counts()

# Sortowanie użytkowników według liczby interakcji od największej do najmniejszej
sorted_users = user_interaction_counts.sort_values(ascending=False)

# Wyświetlenie posortowanych użytkowników
print(sorted_users.head())

user
1807849    279
18730      266
691341     259
1132673    254
650491     245
Name: count, dtype: int64


In [51]:
# # user_interaction_counts = df['user'].value_counts()
# # item_interaction_counts = df['item'].value_counts()

# # # Wybór 20% użytkowników o największej liczbie interakcji
# # top_users = user_interaction_counts.head(int(len(user_interaction_counts) * 0.2))

# # # Wybór 20% przedmiotów o największej liczbie interakcji
# # top_items = item_interaction_counts.head(int(len(item_interaction_counts) * 0.2))

# # Wybór x% użytkowników i przedmiotów do próbkowania
# sample_percent = 0.2  # Możesz zmieniać ten procent w zakresie od 0.01 do 0.5

# # Konwertowanie indeksu na ramkę danych dla użytkowników i przedmiotów
# top_users_df = top_users.reset_index()
# top_users_df.columns = ['user', 'interactions']
# top_items_df = top_items.reset_index()
# top_items_df.columns = ['item', 'interactions']

# # Wybór x% użytkowników
# sampled_users = filtered_df.sample(frac=sample_percent, random_state=42)

# # Wybór x% przedmiotów
# sampled_items = filtered_df.sample(frac=sample_percent, random_state=42)

# # Wyświetlenie wybranych użytkowników i przedmiotów
# print("Wybrane użytkownicy:")
# print(sampled_users.head())
# print("\nWybrane przedmioty:")
# print(sampled_items.head())

In [52]:
# filtered_df = df[df['user'].isin(sampled_users['user']) & df['item'].isin(sampled_items['item'])]

# # Wyświetlenie przefiltrowanej ramki danych
# print(filtered_df.head())

In [53]:
# user_interaction_counts = df.groupby('user')['label'].count()
# item_interaction_counts = df.groupby('item')['label'].count()
# user_mean = user_interaction_counts.mean()
# user_std = user_interaction_counts.std()
# item_mean = item_interaction_counts.mean()
# item_std = item_interaction_counts.std()

# # Ustalenie granicy dla wartości odstających (trzy odchylenia standardowe od średniej)
# user_outlier_threshold = user_mean + 3 * user_std
# item_outlier_threshold = item_mean + 3 * item_std

# # Odrzucenie outlierów dla użytkowników
# users_no_outliers = df.groupby('user').filter(lambda x: x['label'].count() < user_outlier_threshold)

# # Odrzucenie outlierów dla przedmiotów
# items_no_outliers = df.groupby('item').filter(lambda x: x['label'].count() < item_outlier_threshold)

# # Ograniczenie ramki danych do użytkowników i przedmiotów bez outlierów
# df_no_outliers = df[df['user'].isin(users_no_outliers['user'].unique()) & df['item'].isin(items_no_outliers['item'].unique())]

In [54]:
# count_users_before = df['user'].nunique()
# count_items_before = df['item'].nunique()
# count_users = df_no_outliers['user'].nunique()
# print(count_users)
# count_items = df_no_outliers['item'].nunique()
# print(count_items)
# print(count_items/count_users * 100)
# count_items_before/count_users_before * 100

In [55]:
# user_interaction_counts = df_no_outliers.groupby('user')['label'].count()
# item_interaction_counts = df_no_outliers.groupby('item')['label'].count()

# # Obliczenie liczby unikalnych wartości, które chcemy zachować
# n_unique_values_to_keep = int(len(item_interaction_counts) * 0.2)

# # Wybieranie 20% najczęściej występujących unikalnych wartości 'item'
# most_common_values = item_interaction_counts.head(n_unique_values_to_keep).index.tolist()

# # Ograniczenie ramki danych do wierszy zawierających te wartości
# filtered_df = df_no_outliers[df_no_outliers['item'].isin(most_common_values)]

# n_unique_values_to_keep_user = int(len(user_interaction_counts) * 0.2)

# # Wybieranie 20% najczęściej występujących unikalnych wartości 'item'
# most_common_values = user_interaction_counts.head(n_unique_values_to_keep_user).index.tolist()

# # Ograniczenie ramki danych do wierszy zawierających te wartości
# filtered_df = filtered_df[filtered_df['user'].isin(most_common_values)]

In [56]:
# count_users = filtered_df['user'].nunique()
# print(count_users)
# count_items = filtered_df['item'].nunique()
# print(count_items)
# print(count_items/count_users * 100)
# count_items_before/count_users_before * 100

In [57]:
# grouped = filtered_df.groupby(['user', 'item'])['label'].nunique()

# # Znajdowanie indeksów wierszy, które spełniają warunek (tylko jedna unikalna wartość 'event')
# rows_to_drop = grouped[grouped == 1].index

# # Usuwanie wierszy, które spełniają warunek
# filtered_df = filtered_df[~filtered_df.set_index(['user', 'item']).index.isin(rows_to_drop)]

## Podział przefiltrowanej ramki danych na zbiór treningowy i testowy

In [58]:
distinct_users = filtered_df['user'].unique()

num_users_group1 = int(0.7 * distinct_users.shape[0])  # 7%
num_users_group2 = int(0.3 * distinct_users.shape[0])  # 3%

group1_users = random.sample(list(distinct_users), num_users_group1)
remaining_users = list(set(distinct_users) - set(group1_users))
group2_users = random.sample(remaining_users, num_users_group2)
#group3_users = list(set(remaining_users) - set(group2_users))

group1_df = filtered_df[filtered_df['user'].isin(group1_users)]
group2_df = filtered_df[filtered_df['user'].isin(group2_users)]
#group3_df = filtered_df[filtered_df['user'].isin(group3_users)]

In [59]:
# Obliczenie liczby interakcji dla każdego przedmiotu
item_interaction_counts = group1_df['item'].value_counts()

# Obliczenie kwantyla dla top 20% przedmiotów
top_20_threshold = item_interaction_counts.quantile(0.8)

# Wybór przedmiotów, które są w top 20% o największej liczbie interakcji
top_20_items = item_interaction_counts[item_interaction_counts >= top_20_threshold].index

# Przefiltrowanie oryginalnej ramki danych, aby pozostawić tylko wybranych użytkowników
group1_df = group1_df[group1_df['item'].isin(top_20_items)]

In [60]:
# unique_users_count = group1_df['user'].nunique()
# unique_items_count = group1_df['item'].nunique()

# Wybór x% unikatowych użytkowników
#sampled_users = filtered_df.drop_duplicates('user').sample(frac=sample_percent, random_state=2024)

# # Wybór x% unikatowych przedmiotów
# sampled_items = group1_df.drop_duplicates('item').sample(frac=0.2, random_state=2024)

# Przefiltrowanie przefiltrowanej ramki danych, aby pozostawić tylko wybrane użytkowniki
#filtered_df_users = filtered_df[filtered_df['user'].isin(sampled_users['user'])]

# # Przefiltrowanie wynikowej ramki danych, aby pozostawić tylko wybrane przedmioty
# group1_df = group1_df[group1_df['item'].isin(sampled_items['item'])]

# user_interaction_counts = group1_df.groupby('user')['label'].count()
# item_interaction_counts = group1_df.groupby('item')['label'].count()

# # Obliczenie liczby unikalnych wartości, które chcemy zachować
# n_unique_values_to_keep = int(len(item_interaction_counts) * 0.2)

# # Wybieranie 20% najczęściej występujących unikalnych wartości 'item'
# most_common_values = item_interaction_counts.head(n_unique_values_to_keep).index.tolist()

# # Ograniczenie ramki danych do wierszy zawierających te wartości
# filtered_g1 = group1_df[group1_df['item'].isin(most_common_values)]

# n_unique_values_to_keep_user = int(len(user_interaction_counts) * 0.8)

# # Wybieranie 20% najczęściej występujących unikalnych wartości 'item'
# most_common_values = user_interaction_counts.head(n_unique_values_to_keep_user).index.tolist()

# # Ograniczenie ramki danych do wierszy zawierających te wartości
# filtered_g1 = filtered_g1[filtered_g1['user'].isin(most_common_values)]

In [61]:
count_users = group1_df['user'].nunique()
print(count_users)
count_items = group1_df['item'].nunique()
print(count_items)
print(count_items/count_users * 100)
count_items_before/count_users_before * 100

74829
1490
1.9912066177551484


5.624947283659725

In [62]:
user_interaction_counts = group1_df['user'].value_counts()

# Sortowanie użytkowników według liczby interakcji od największej do najmniejszej
sorted_users = user_interaction_counts.sort_values(ascending=False)

# Wyświetlenie posortowanych użytkowników
print(sorted_users.head())

user
3268166    179
2962358    175
691341     164
18730      163
1132673    151
Name: count, dtype: int64


In [63]:
# def split_data_by_user_quantile(df, quant):
# #     # Group the data by user
#      grouped = df.groupby('user')

# #     # Initialize empty lists to store data below and above quantile threshold for each user
#      below_quantile_list = []
#      above_quantile_list = []

# #     # Iterate over each user group
#      for user, group_data in grouped:
#          # Calculate the 70th quantile of the timestamp column for the current user
#          quantile_user = group_data['time'].quantile(quant)

# #         # Split the data for the current user into two sets based on the quantile threshold
#          below_quantile = group_data[group_data['time'] <= quantile_user]
#          above_quantile = group_data[group_data['time'] > quantile_user]

# #         # Append the sets to the respective lists
#          below_quantile_list.append(below_quantile)
#          above_quantile_list.append(above_quantile)

# #     # Concatenate the sets for all users into DataFrames
#      below_quantile_df = pd.concat(below_quantile_list)
#      above_quantile_df = pd.concat(above_quantile_list)

#      return below_quantile_df, above_quantile_df

# # # Split data for each group based on user-specific quantile threshold
# group1_below_user_quantile, group1_above_user_quantile = split_data_by_user_quantile(filtered_g1, 0.7)
# # group2_below_user_quantile, group2_above_user_quantile = split_data_by_user_quantile(group2_df, 0.7)

# # # Display the shapes of the resulting sets
# print("Group 1 - Below or equal  70th user-specific quantile:", group1_below_user_quantile.shape)
# print("Group 1 - Above to 70th user-specific quantile:", group1_above_user_quantile.shape)
# #print("Group 2 - Below or equal 70th user-specific quantile:", group2_below_user_quantile.shape)
# #print("Group 2 - Above to 70th user-specific quantile:", group2_above_user_quantile.shape)

In [64]:
# print(group1_below_user_quantile['item'].nunique())
# group1_above_user_quantile['item'].nunique()

In [65]:
# unique_items_b = set(group1_below_user_quantile['item'].unique())
# unique_items_a = set(group1_above_user_quantile['item'].unique())

# unique_items_only_in_a = unique_items_a.difference(unique_items_b)
# unique_items_only_in_b = unique_items_b.difference(unique_items_a)

# print(len(unique_items_only_in_a))
# print(len(unique_items_only_in_b))

Implementujemy kod z biblioteki LibRecommender dzielący dane na zbiór treningowy i ewaluacyjny.

In [67]:
# train_data = group1_below_user_quantile
# eval_data = group1_above_user_quantile
train_data, eval_data = split_by_ratio_chrono(filtered_df,test_size=0.2)
train_data, eval_data = split_by_ratio_chrono(group1_df,test_size=0.2)
train_data, data_info = DatasetPure.build_trainset(train_data)
eval_data = DatasetPure.build_evalset(eval_data)
#test_data = DatasetPure.build_testset(test_data)
print(data_info)

n_users: 130904, n_items: 7425, data density: 0.1203 %


In [68]:
lightgcn = LightGCN(
    task="ranking",
    data_info=data_info,
    loss_type="bpr",
    embed_size=32,
    n_epochs=3,
    lr=1e-2,
    batch_size=2048,
    num_neg=1,
    device="cuda"
)

In [69]:
lightgcn.fit(
    train_data,
    neg_sampling=True,
    verbose=2,
    eval_data=eval_data,
    metrics=["loss", "precision", "recall", "ndcg", "map"]
)

Training start time: 2024-03-03 17:50:51


train: 100%|█████████████████████████████████████████████████████████████████████████| 571/571 [06:26<00:00,  1.48it/s]


Epoch 1 elapsed: 386.741s
	 train_loss: 0.1617


eval_pointwise: 100%|█████████████████████████████████████████████████████████████████| 68/68 [00:00<00:00, 523.91it/s]
C:\Users\marci\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:2910: FutureWarning: Setting the eps parameter is deprecated and will be removed in 1.5. Instead eps will always havea default value of `np.finfo(y_pred.dtype).eps`.
  warnings.warn(
eval_listwise: 100%|█████████████████████████████████████████████████████████| 101083/101083 [00:16<00:00, 5976.60it/s]


	 eval log_loss: 0.5639
	 eval precision@10: 0.0186
	 eval recall@10: 0.0949
	 eval ndcg@10: 0.0876
	 eval map@10: 0.0623


...

	 eval log_loss: 0.6064
	 eval precision@10: 0.0204
	 eval recall@10: 0.1045
	 eval ndcg@10: 0.0955
	 eval map@10: 0.0679


train: 100%|█████████████████████████████████████████████████████████████████████████| 571/571 [06:47<00:00,  1.40it/s]


Epoch 3 elapsed: 407.854s
	 train_loss: 0.0514


eval_pointwise: 100%|█████████████████████████████████████████████████████████████████| 68/68 [00:00<00:00, 455.70it/s]
C:\Users\marci\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:2910: FutureWarning: Setting the eps parameter is deprecated and will be removed in 1.5. Instead eps will always havea default value of `np.finfo(y_pred.dtype).eps`.
  warnings.warn(
eval_listwise: 100%|█████████████████████████████████████████████████████████| 101083/101083 [00:17<00:00, 5805.58it/s]


	 eval log_loss: 0.6395
	 eval precision@10: 0.0214
	 eval recall@10: 0.1095
	 eval ndcg@10: 0.1001
	 eval map@10: 0.0714


In [70]:
lightgcn.fit(
    train_data,
    neg_sampling=True,
    verbose=2,
    eval_data=eval_data,
    metrics=["loss", "precision", "recall", "ndcg", "map"],
    k=5
)

Training start time: 2024-03-03 18:12:39


train: 100%|█████████████████████████████████████████████████████████████████████████| 571/571 [06:50<00:00,  1.39it/s]


Epoch 1 elapsed: 410.222s
	 train_loss: 0.0297


eval_pointwise: 100%|█████████████████████████████████████████████████████████████████| 68/68 [00:00<00:00, 472.51it/s]
C:\Users\marci\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:2910: FutureWarning: Setting the eps parameter is deprecated and will be removed in 1.5. Instead eps will always havea default value of `np.finfo(y_pred.dtype).eps`.
  warnings.warn(
eval_listwise: 100%|█████████████████████████████████████████████████████████| 101083/101083 [00:17<00:00, 5863.04it/s]


	 eval log_loss: 0.6739
	 eval precision@5: 0.0263
	 eval recall@5: 0.0683
	 eval ndcg@5: 0.0793
	 eval map@5: 0.0645


...

	 eval log_loss: 0.7116
	 eval precision@5: 0.0262
	 eval recall@5: 0.0677
	 eval ndcg@5: 0.0786
	 eval map@5: 0.0639


train: 100%|█████████████████████████████████████████████████████████████████████████| 571/571 [06:30<00:00,  1.46it/s]


Epoch 3 elapsed: 390.984s
	 train_loss: 0.0169


eval_pointwise: 100%|█████████████████████████████████████████████████████████████████| 68/68 [00:00<00:00, 477.74it/s]
C:\Users\marci\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:2910: FutureWarning: Setting the eps parameter is deprecated and will be removed in 1.5. Instead eps will always havea default value of `np.finfo(y_pred.dtype).eps`.
  warnings.warn(
eval_listwise: 100%|█████████████████████████████████████████████████████████| 101083/101083 [00:17<00:00, 5778.93it/s]


	 eval log_loss: 0.7486
	 eval precision@5: 0.0260
	 eval recall@5: 0.0672
	 eval ndcg@5: 0.0787
	 eval map@5: 0.0642


In [71]:
lightgcn.fit(
    train_data,
    neg_sampling=True,
    verbose=2,
    eval_data=eval_data,
    metrics=["loss", "precision", "recall", "ndcg", "map"],
    k=2
)

Training start time: 2024-03-03 18:33:35


train: 100%|█████████████████████████████████████████████████████████████████████████| 571/571 [06:46<00:00,  1.41it/s]


Epoch 1 elapsed: 406.323s
	 train_loss: 0.0109


eval_pointwise: 100%|█████████████████████████████████████████████████████████████████| 68/68 [00:00<00:00, 467.42it/s]
C:\Users\marci\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:2910: FutureWarning: Setting the eps parameter is deprecated and will be removed in 1.5. Instead eps will always havea default value of `np.finfo(y_pred.dtype).eps`.
  warnings.warn(
eval_listwise: 100%|█████████████████████████████████████████████████████████| 101083/101083 [00:16<00:00, 5980.20it/s]


	 eval log_loss: 0.7876
	 eval precision@2: 0.0315
	 eval recall@2: 0.0330
	 eval ndcg@2: 0.0518
	 eval map@2: 0.0483


...

	 eval log_loss: 0.8303
	 eval precision@2: 0.0308
	 eval recall@2: 0.0323
	 eval ndcg@2: 0.0507
	 eval map@2: 0.0472


train: 100%|█████████████████████████████████████████████████████████████████████████| 571/571 [06:29<00:00,  1.47it/s]


Epoch 3 elapsed: 389.051s
	 train_loss: 0.0059


eval_pointwise: 100%|█████████████████████████████████████████████████████████████████| 68/68 [00:00<00:00, 438.80it/s]
C:\Users\marci\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:2910: FutureWarning: Setting the eps parameter is deprecated and will be removed in 1.5. Instead eps will always havea default value of `np.finfo(y_pred.dtype).eps`.
  warnings.warn(
eval_listwise: 100%|█████████████████████████████████████████████████████████| 101083/101083 [00:17<00:00, 5686.65it/s]


	 eval log_loss: 0.8716
	 eval precision@2: 0.0304
	 eval recall@2: 0.0317
	 eval ndcg@2: 0.0501
	 eval map@2: 0.0468


Wszystko poniżej tego kawałka kodu to zabawy w zmianę funkcji straty, zmiana parametrów itd itp.

In [ ]:
HAMUJ SIĘ

In [33]:
# evaluate(
#     model=lightgcn,
#     data=group2_df,
#     neg_sampling=True,
#     metrics=["loss", "precision", "recall", "ndcg", "map"],
#     k=10
# )

In [ ]:
lightgcn = LightGCN(
    task="ranking",
    data_info=data_info,
    loss_type="cross_entropy",
    embed_size=32,
    n_epochs=5,
    lr=0.001,
    batch_size=512,
    num_neg=1,
    device="cuda"
)

lightgcn.fit(
    train_data,
    neg_sampling=True,
    verbose=2,
    eval_data=eval_data,
    metrics=["loss", "precision", "recall", "ndcg", "map"]
)

In [ ]:
lightgcn = LightGCN(
    task="ranking",
    data_info=data_info,
    loss_type="cross_entropy",
    embed_size=64,
    n_epochs=5,
    lr=1e-2,
    batch_size=2048,
    num_neg=1,
    device="cuda"
)

lightgcn.fit(
    train_data,
    neg_sampling=True,
    verbose=2,
    eval_data=eval_data,
    metrics=["loss", "precision", "recall", "ndcg", "map"]
)

In [ ]:
lightgcn = LightGCN(
    task="ranking",
    data_info=data_info,
    loss_type="focal",
    embed_size=64,
    n_epochs=5,
    lr=1e-2,
    batch_size=2048,
    num_neg=1,
    device="cuda"
)

lightgcn.fit(
    train_data,
    neg_sampling=True,
    verbose=2,
    eval_data=eval_data,
    metrics=["loss", "precision", "recall", "ndcg", "map"]
)

In [ ]:
lightgcn = LightGCN(
    task="ranking",
    data_info=data_info,
    loss_type="max_margin",
    embed_size=64,
    n_epochs=5,
    lr=1e-2,
    batch_size=2048,
    num_neg=1,
    device="cuda"
)

lightgcn.fit(
    train_data,
    neg_sampling=True,
    verbose=2,
    eval_data=eval_data,
    metrics=["loss", "precision", "recall", "ndcg", "map"]
)